In [1]:
from dotenv import load_dotenv
import os
import csv, json
import pandas as pd
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import load_prompt


data_path = 'D:\\00_projects\GAS5_FP\\Crawling\\openapi\\data'
load_dotenv()

True

In [2]:
df = pd.read_json(os.path.join(data_path,'e약은요_약성포함.json'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50494 entries, 0 to 50493
Data columns (total 44 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   업체명_x         50494 non-null  object 
 1   제품명           50494 non-null  object 
 2   품목기준코드        50494 non-null  int64  
 3   효능            50285 non-null  object 
 4   사용법           50434 non-null  object 
 5   주의사항_경고       11927 non-null  object 
 6   주의사항          50447 non-null  object 
 7   상호작용          37160 non-null  object 
 8   부작용           47650 non-null  object 
 9   보관법           50416 non-null  object 
 10  공개일자          50481 non-null  object 
 11  수정일자          50494 non-null  object 
 12  낱알이미지         34980 non-null  object 
 13  사업자등록번호       50494 non-null  int64  
 14  한글상품명         50487 non-null  object 
 15  업체명_y         50487 non-null  object 
 16  약품규격          50487 non-null  object 
 17  제품총수량         50487 non-null  float64
 18  제형구분          34622 non-null  o

In [3]:
dur = pd.read_csv(
    os.path.join(data_path, '성분DUR', 'OpenData_PotOpenDurIngr_AC20241017(병용금기).csv'),
    on_bad_lines='skip',  # 문제가 있는 줄을 건너뜁니다.
    quoting=csv.QUOTE_NONE,  # 따옴표를 무시하도록 설정합니다.
    encoding='utf-8',  # 파일의 인코딩이 utf-8일 경우
)

dur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1421 entries, 0 to 1420
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   DUR일련번호       1421 non-null   int64  
 1   DUR유형         1421 non-null   object 
 2   단일복합구분코드      1421 non-null   object 
 3   DUR성분코드       1421 non-null   object 
 4   DUR성분명영문      1421 non-null   object 
 5   DUR성분명        1421 non-null   object 
 6   복합제           1421 non-null   object 
 7   관계성분          1421 non-null   object 
 8   약효분류코드        1421 non-null   object 
 9   효능군           1421 non-null   object 
 10  고시일자          1421 non-null   int64  
 11  금기내용          1421 non-null   object 
 12  제형            1421 non-null   object 
 13  연령기준          1421 non-null   object 
 14  최대투여기간        1421 non-null   object 
 15  1일최대용량        1421 non-null   object 
 16  등급            1421 non-null   object 
 17  병용금기단일복합구분코드  1421 non-null   object 
 18  병용금기DUR성분코드   1421 non-null 

- 다음의 데이터를 활용 : 
	- user_id -- 유저 식별.
	- 약물 정보
		- 제품명
			- 성분명
			- DUR
				- 금기 내용
				- 병용금기DUR성분명
		- 효능
		- 사용법
		- 주의사항_경고
		- 주의사항
		- 상호작용
		- 부작용
		- 보관법
		- 공개일자
		- 수정일자
		- 투여

In [4]:
'''
해당 부분은 실제 구현
1. 복약 리스트(약물 식별 변호 - 품목기준코드, 일반명코드)를 받는다
2. DB에서 약물 정보, 품목 및 성분 DUR 정보를 조회한다.
'''

df_medication_info = df[
    [
        '제품명', 
        '일반명', 
        '효능', 
        '사용법', 
        '주의사항_경고', 
        '주의사항', 
        '상호작용', 
        '부작용', 
        '보관법', 
        '투여'
        ]
    ]

df_medication_info.head(2)

,제품명,일반명,효능,사용법,주의사항_경고,주의사항,상호작용,부작용,보관법,투여
0,활명수,atractylodes rhizome,"이 약은 식욕감퇴(식욕부진), 위부팽만감, 소화불량, 과식, 체함, 구역, 구토에 ...","만 15세 이상 및 성인은 1회 1병(75 mL), 만 11세이상~만 15세미만은 ...",None,만 3개월 미만의 젖먹이는 이 약을 복용하지 마십시오.\n\n이 약을 복용하기 전에...,None,None,습기와 빛을 피해 실온에서 보관하십시오.\n\n어린이의 손이 닿지 않는 곳에 보관하...,내복
1,활명수,capsicum tinc.,"이 약은 식욕감퇴(식욕부진), 위부팽만감, 소화불량, 과식, 체함, 구역, 구토에 ...","만 15세 이상 및 성인은 1회 1병(75 mL), 만 11세이상~만 15세미만은 ...",None,만 3개월 미만의 젖먹이는 이 약을 복용하지 마십시오.\n\n이 약을 복용하기 전에...,None,None,습기와 빛을 피해 실온에서 보관하십시오.\n\n어린이의 손이 닿지 않는 곳에 보관하...,내복


In [5]:

# 중복 행 제거
df_unique = df_medication_info.drop_duplicates(subset=['일반명'])

# 'Acetaminophen'이 포함된 행 필터링
sample = df_unique[df_unique['제품명'].str.contains('돔페리돈', case=False, na=False)]
sample

,제품명,일반명,효능,사용법,주의사항_경고,주의사항,상호작용,부작용,보관법,투여
3814,멕시롱액(돔페리돈),domperidone,"이 약은 오심, 구토 증상의 완화에 사용합니다.","성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 ...",None,"이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공)...","강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래...","구강건조증, 성욕소실, 불안, 졸음, 두퉁, 설사, 발진, 가려움, 유즙분비과다, ...",빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,내복


In [6]:
'''
성분을 바탕으로 DUR 정보 조회
'''

sample_dur = dur[dur['DUR성분명영문'].str.contains('|'.join(sample['일반명']), case=False, na=False)].drop_duplicates(subset=['병용금기DUR성분명'])

# 금기내용과 병용금기DUR성분명, DUR성분명영문을 포함하여 그룹화
sample_dur = sample_dur.groupby('금기내용').agg({
    '병용금기DUR성분명': lambda x: ', '.join(x),
    'DUR성분명영문': 'first'  # 첫 번째 값을 취함 (원하는 방식으로 조정 가능)
}).reset_index()

sample_dur


,금기내용,병용금기DUR성분명,DUR성분명영문
0,"""QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)...",Ketoconazole,Domperidone
1,QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)가능성,"Chlorpromazine, Fluconazole, Metoclopramide, P...",Domperidone
2,QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)가...,"Posaconazole, Telithromycin, Voriconazole, Ata...",Domperidone
3,강력한CYP억제제가돔페리돈의대사를감소시켜|돔페리돈의영향(이상반응포함)증대가능성,"Nelfinavir, Darunavir, Indinavir",Domperidone


In [7]:
'''
sample에 병용금기 DUR 업데이트
'''

sample = sample.merge(sample_dur[['DUR성분명영문', '병용금기DUR성분명', '금기내용']], 
                       left_on=sample['일반명'].str.lower(), 
                       right_on=sample_dur['DUR성분명영문'].str.lower(),  
                       how='left')

sample

,key_0,제품명,일반명,효능,사용법,주의사항_경고,주의사항,상호작용,부작용,보관법,투여,DUR성분명영문,병용금기DUR성분명,금기내용
0,domperidone,멕시롱액(돔페리돈),domperidone,"이 약은 오심, 구토 증상의 완화에 사용합니다.","성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 ...",None,"이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공)...","강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래...","구강건조증, 성욕소실, 불안, 졸음, 두퉁, 설사, 발진, 가려움, 유즙분비과다, ...",빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,내복,Domperidone,Ketoconazole,"""QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)..."
1,domperidone,멕시롱액(돔페리돈),domperidone,"이 약은 오심, 구토 증상의 완화에 사용합니다.","성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 ...",None,"이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공)...","강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래...","구강건조증, 성욕소실, 불안, 졸음, 두퉁, 설사, 발진, 가려움, 유즙분비과다, ...",빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,내복,Domperidone,"Chlorpromazine, Fluconazole, Metoclopramide, P...",QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)가능성
2,domperidone,멕시롱액(돔페리돈),domperidone,"이 약은 오심, 구토 증상의 완화에 사용합니다.","성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 ...",None,"이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공)...","강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래...","구강건조증, 성욕소실, 불안, 졸음, 두퉁, 설사, 발진, 가려움, 유즙분비과다, ...",빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,내복,Domperidone,"Posaconazole, Telithromycin, Voriconazole, Ata...",QTc연장효과증대로심각한위험(TorsadedePointes|심각한심실성부정맥포함)가...
3,domperidone,멕시롱액(돔페리돈),domperidone,"이 약은 오심, 구토 증상의 완화에 사용합니다.","성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 ...",None,"이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공)...","강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래...","구강건조증, 성욕소실, 불안, 졸음, 두퉁, 설사, 발진, 가려움, 유즙분비과다, ...",빛을 피해 실온에서 보관하십시오.어린이의 손이 닿지 않는 곳에 보관하십시오.,내복,Domperidone,"Nelfinavir, Darunavir, Indinavir",강력한CYP억제제가돔페리돈의대사를감소시켜|돔페리돈의영향(이상반응포함)증대가능성


In [8]:
''' 
검색 결과 병합. (단일약물에 대해.)
'''
 
# 공통 정보를 저장
common_info = {
    "key": sample.iloc[0]["key_0"],
    "제품명": sample.iloc[0]["제품명"],
    "효능": sample.iloc[0]["효능"],
    "사용법": sample.iloc[0]["사용법"],
    "주의사항_경고": sample.iloc[0]["주의사항_경고"],
    "주의사항": sample.iloc[0]["주의사항"],
    "상호작용": sample.iloc[0]["상호작용"],
    "부작용": sample.iloc[0]["부작용"],
    "보관법": sample.iloc[0]["보관법"],
    "투여": sample.iloc[0]["투여"],
    "하위정보": []  # 하위 정보 리스트 초기화
}

# 하위 JSON 구성
for index, row in sample.iterrows():
    sub_info = {
        "일반명": row["일반명"],
        "병용금기DUR성분명": row["병용금기DUR성분명"],
        "금기내용" : row["금기내용"]
    }
    common_info["하위정보"].append(sub_info)

# 최종 JSON 객체 생성
medi_info = common_info

# JSON 형식으로 출력
medi_infos = json.dumps(medi_info, ensure_ascii=False, indent=2)
print(medi_infos)

{
  "key": "domperidone",
  "제품명": "멕시롱액(돔페리돈)",
  "효능": "이 약은 오심, 구토 증상의 완화에 사용합니다.",
  "사용법": "성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 5 mg)을 1일 3회 식전에 복용합니다.1일 최대 복용량은 30 mg이며, 일반적으로 1주일을 초과하여 사용하지 않도록 합니다.",
  "주의사항_경고": null,
  "주의사항": "이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공), 프로락틴분비종양, 강력한 CYP3A4 저해제를 병용투여중인 환자, 중등도 및 중증의 간장애, 임부 또는 임신하고 있을 가능성이 있는 여성, QTc 간격을 지연시키는 약물을 병용투여중인 환자, 전해질 불균형(저칼륨혈증, 고칼륨혈증, 저마그네슘혈증)을 보이는 환자, 심부전과 같은 심장 기저질환 환자, 갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등의 유전적인 문제가 있는 환자는 이 약을 복용하지 마십시오.이 약을 복용하기 전에 소아, 신장애 또는 경증 간장애 환자, 고령자, 심장질환 환자 또는 경험자, 수유부, 유방암 경험자, 카라멜 성분에 과민증 환자는 의사 또는 약사와 상의하십시오.이 약 복용 후 졸음 및 어지러움이 나타날 수 있으므로 운전 및 기계조작 시 주의하십시오.",
  "상호작용": "강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래리트로마이신, 에리트로마이신, 텔리트로마이신, 암프레나비르, 아타자나비르, 포삼프레나비르, 인디나비르, 넬피나비르, 리토나비르, 사퀴나비르), QTc와 같은 심장전도 간격을 지연시키는 약물(디소피라미드, 퀴니딘, 아미오다론, 드로네다론, 소타롤, 도페틸리드, 이부틸리드, 할로페리돌, 피모지드, 설틴돌, 시탈로프람, 에시탈로프람, 레보플록사신, 목시플록사신, 에리트로마이신, 스피라마이신, 펜타미딘, 할로판트린, 루메판트린, 돌라세트론, 

In [9]:
gpt = ChatOpenAI(
    temperature=0,
    model_name="gpt-4o",  # 모델명
)

In [10]:
def restruct(medi_infos:dict, llm):
        prompt = load_prompt("prompts/restruct.yaml", encoding="utf-8")
        chain = prompt | llm
        return chain.invoke(medi_infos).content        

In [11]:
restructed = restruct(medi_infos, gpt)
print(restructed)

```json
[
  {
    "key": "domperidone",
    "제품명": "멕시롱액(돔페리돈)",
    "효능": "이 약은 오심, 구토 증상의 완화에 사용합니다.",
    "사용법": "성인 및 청소년(12세 이상, 35 kg 이상)은 1회 10 mg(레보도파 투여시 5 mg)을 1일 3회 식전에 복용합니다. 1일 최대 복용량은 30 mg이며, 일반적으로 1주일을 초과하여 사용하지 않도록 합니다.",
    "주의사항_경고": null,
    "주의사항": "이 약에 과민증 환자, 위운동 촉진이 위험한 환자(위장관출혈, 기계적폐쇄증, 천공), 프로락틴분비종양, 강력한 CYP3A4 저해제를 병용투여중인 환자, 중등도 및 중증의 간장애, 임부 또는 임신하고 있을 가능성이 있는 여성, QTc 간격을 지연시키는 약물을 병용투여중인 환자, 전해질 불균형(저칼륨혈증, 고칼륨혈증, 저마그네슘혈증)을 보이는 환자, 심부전과 같은 심장 기저질환 환자, 갈락토오스 불내성, Lapp 유당분해효소 결핍증 또는 포도당-갈락토오스 흡수장애 등의 유전적인 문제가 있는 환자는 이 약을 복용하지 마십시오. 이 약을 복용하기 전에 소아, 신장애 또는 경증 간장애 환자, 고령자, 심장질환 환자 또는 경험자, 수유부, 유방암 경험자, 카라멜 성분에 과민증 환자는 의사 또는 약사와 상의하십시오. 이 약 복용 후 졸음 및 어지러움이 나타날 수 있으므로 운전 및 기계조작 시 주의하십시오.",
    "상호작용": "강력한 CYP3A4 저해제(플루코나졸, 이트라코나졸, 케토코나졸, 보리코나졸, 클래리트로마이신, 에리트로마이신, 텔리트로마이신, 암프레나비르, 아타자나비르, 포삼프레나비르, 인디나비르, 넬피나비르, 리토나비르, 사퀴나비르), QTc와 같은 심장전도 간격을 지연시키는 약물(디소피라미드, 퀴니딘, 아미오다론, 드로네다론, 소타롤, 도페틸리드, 이부틸리드, 할로페리돌, 피모지드, 설틴돌, 시탈로프람, 에시탈로프람, 레보플록사신, 목시플록사신, 에리트로마이신, 스피라마이신

In [12]:
def summurize(context:str, llm):
    prompt = load_prompt("prompts/summarize.yaml", encoding="utf-8")
    chain = prompt | llm
    return chain.invoke(context).content    

In [13]:
summary = summurize(restructed, gpt)
print(summary)

멕시롱액(돔페리돈)은 오심과 구토 증상을 완화하는 데 사용되는 약입니다. 이 약에 대해 자주 묻는 질문과 그에 대한 답변을 통해 중요한 정보를 제공하겠습니다.

1. **이 약은 어떻게 복용하나요?**
   - 성인과 12세 이상, 체중 35kg 이상의 청소년은 1회 10mg을 하루 3번 식전에 복용합니다. 레보도파를 함께 복용하는 경우에는 5mg을 복용합니다. 하루 최대 복용량은 30mg이며, 일반적으로 1주일을 초과하여 사용하지 않습니다.

2. **이 약을 복용하면 안 되는 경우가 있나요?**
   - 이 약에 알레르기가 있는 분, 위장관 출혈이나 폐쇄증, 천공이 있는 분, 프로락틴 분비 종양이 있는 분, 강력한 CYP3A4 저해제를 복용 중인 분, 중등도 및 중증의 간장애가 있는 분, 임신 중이거나 임신 가능성이 있는 여성, QTc 간격을 지연시키는 약물을 복용 중인 분, 전해질 불균형이 있는 분, 심부전과 같은 심장 질환이 있는 분, 갈락토오스 불내성 등의 유전적 문제가 있는 분은 이 약을 복용하지 말아야 합니다.

3. **복용 시 주의해야 할 점은 무엇인가요?**
   - 소아, 신장애 또는 경증 간장애 환자, 고령자, 심장질환 환자, 수유부, 유방암 경험자, 카라멜 성분에 과민증이 있는 분은 의사나 약사와 상의 후 복용해야 합니다. 복용 후 졸음이나 어지러움이 나타날 수 있으므로 운전이나 기계 조작 시 주의가 필요합니다.

4. **다른 약과 함께 복용해도 되나요?**
   - 강력한 CYP3A4 저해제나 QTc 간격을 지연시키는 약물과 함께 복용하지 말아야 합니다. 페노티아진계, 부티로페논계, 라우올피아 알칼로이드 제제, 디기탈리스제제, 항콜린제, 도파민효능약, 제산제, 위산분비억제제, 항무스카린 약물, 아편양 진통제와 함께 복용 시에는 의사나 약사와 상의해야 합니다.

5. **부작용이 있나요?**
   - 구강건조증, 성욕소실, 불안, 졸음, 두통, 설사, 발진, 가려움, 유즙분비과다, 유방통증, 무력증, 아나필락시스 반응, 초조, 신경질,

In [14]:
# 추후 요약 작성시에 verifier의 결과를 통합해 요약을 재생성하는 방식으로 사용 가능할듯.
def verifier(original_content:str, summary:str, llm):
    prompt = load_prompt("prompts/verify.yaml", encoding="utf-8")
    chain = prompt | llm
    return chain.invoke({'original_content' : original_content, 'summary':summary}).content

In [15]:
verification = verifier(restructed, summary, gpt)
print(verification)

1. **일관성 확인**:
   - 요약은 원본 내용의 주요 사항과 세부 사항을 대체로 정확하게 반영하고 있습니다. 복용 방법, 금기 사항, 주의 사항, 상호작용, 부작용, 보관 방법에 대한 정보가 포함되어 있습니다.

2. **누락된 정보**:
   - 원본 내용에 있는 "내복"이라는 투여 방법이 요약에 포함되어 있지 않습니다.
   - "하위정보" 섹션에 있는 병용금기 DUR 성분명과 금기 내용이 요약에 포함되어 있지 않습니다. 이는 특정 약물과의 상호작용에 대한 구체적인 정보를 제공합니다.

3. **오해의 소지가 있는 표현**:
   - 요약에는 오해의 소지가 있는 표현은 없습니다. 다만, "다른 약과 함께 복용해도 되나요?" 섹션에서 상호작용에 대한 구체적인 약물 목록이 생략되어 있어, 원본의 세부적인 상호작용 정보를 충분히 전달하지 못할 수 있습니다.

4. **전체 평가**:
   - 요약은 원본 내용의 주요 정보를 대체로 충실하게 반영하고 있으며, 복용 방법, 금기 사항, 주의 사항, 상호작용, 부작용, 보관 방법에 대한 정보를 잘 전달하고 있습니다. 그러나 "하위정보" 섹션의 구체적인 병용금기 DUR 성분명과 금기 내용이 누락되어 있어, 상호작용에 대한 세부적인 정보가 부족합니다. 이러한 점을 보완한다면 요약은 더욱 완전한 형태가 될 것입니다.


In [16]:
''' 
생성된 정리본과 요약본을 user_id를 기준으로 DB에 저장
'''

' \n생성된 정리본과 요약본을 user_id를 기준으로 DB에 저장\n'